In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import datetime


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-1/test.csv
/kaggle/input/covid19-global-forecasting-week-1/train.csv
/kaggle/input/covid19-global-forecasting-week-1/submission.csv


In [2]:
#Defining the dataset
train=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/train.csv')
test=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/test.csv')
sub=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/submission.csv')

Data visualization


In [3]:
#Unique data
country=pd.unique(train['Country/Region'])
print('Number of countries',len(country))
date=pd.unique(train['Date'])
print('number of days',len(date))

Number of countries 163
number of days 61


In [4]:
temp = train.groupby(['Country/Region', 'Province/State'])['ConfirmedCases', 'Fatalities'].max()
temp.head(20)

ConfirmedCases  Fatalities
Country/Region Province/State                                          
Australia      Australian Capital Territory            19.0         0.0
               From Diamond Princess                    8.0         0.0
               New South Wales                        533.0         6.0
               Northern Territory                       3.0         0.0
               Queensland                             221.0         0.0
               South Australia                        100.0         0.0
               Tasmania                                22.0         0.0
               Victoria                               296.0         0.0
               Western Australia                      120.0         1.0
Canada         Alberta                                259.0         1.0
               British Columbia                       424.0        10.0
               Grand Princess                          13.0         0.0
               Manitoba                                19.0         0.0
               New Brunswick                           17.0         0.0
               Newfoundland and Labrador                6.0         0.0
               Nova Scotia                              0.0         0.0
               Ontario                                425.0         5.0
               Prince Edward Island                     2.0         0.0
               Quebec                                 219.0         5.0
               Saskatchewan                            52.0         0.0

In [5]:
temp1=train.groupby('Date')['ConfirmedCases','Fatalities'].sum().reset_index()
temp1=temp1[temp1['Date']==max(temp1['Date'])].reset_index(drop=True)
temp1.style.background_gradient(cmap='Accent')

,Date,ConfirmedCases,Fatalities
0,2020-03-22,332193,14588


In [6]:
train_grouped = train.groupby('Country/Region')['ConfirmedCases', 'Fatalities'].sum().reset_index()

In [7]:
temp_f = train_grouped.sort_values(by='ConfirmedCases', ascending=False)
temp_f = temp_f.reset_index(drop=True)
temp_f.style.background_gradient(cmap='Reds')

,Country/Region,ConfirmedCases,Fatalities
0,China,3.34206e+06,114573
1,Italy,438821,33959
2,Iran,231132,13124
3,"Korea, South",172802,1439
4,Spain,157432,7345
5,Germany,136101,395
6,US,122478,1727
7,France,102321,2859
8,Switzerland,39168,393
9,United Kingdom,29609,1100


In [8]:
# Confirmed and death in map

fig = px.choropleth(train_grouped, locations="Country/Region", 
                    locationmode='country names', color="ConfirmedCases", 
                    hover_name="Fatalities", range_color=[1,7000], 
                    color_continuous_scale="aggrnyl", 
                    title='Confirmed and Death case')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [9]:
formated_gdf = train.groupby(['Date', 'Country/Region'])['ConfirmedCases','Fatalities'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')
formated_gdf['size'] = formated_gdf['ConfirmedCases'].pow(0.3)

fig2 = px.scatter_geo(formated_gdf, locations="Country/Region", locationmode='country names', 
                     color="ConfirmedCases", size='size', hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf['ConfirmedCases'])+2], 
                     projection="natural earth", animation_frame="Date", 
                     title='Spread over time')
fig2.update(layout_coloraxis_showscale=False)
fig2.show()

In [10]:
locations = list(set([(test.loc[i, "Province/State"], test.loc[i, "Country/Region"]) for i in test.index]))
locations

[('Macau', 'China'),
 (nan, 'Ghana'),
 (nan, 'Andorra'),
 ('Minnesota', 'US'),
 ('Jiangsu', 'China'),
 ('Florida', 'US'),
 ('Hainan', 'China'),
 (nan, 'Malta'),
 ('Maine', 'US'),
 ('California', 'US'),
 ('Jilin', 'China'),
 (nan, 'Hungary'),
 (nan, 'Greece'),
 (nan, 'Armenia'),
 ('Zhejiang', 'China'),
 ('Connecticut', 'US'),
 (nan, 'Taiwan*'),
 ('Montana', 'US'),
 (nan, 'Pakistan'),
 (nan, 'Barbados'),
 ('Gansu', 'China'),
 (nan, 'Indonesia'),
 (nan, 'Kosovo'),
 (nan, 'Eswatini'),
 ('Virgin Islands', 'US'),
 (nan, 'Tunisia'),
 (nan, 'Ethiopia'),
 ('Grand Princess', 'US'),
 (nan, 'North Macedonia'),
 (nan, 'Vietnam'),
 ('Guadeloupe', 'France'),
 (nan, "Cote d'Ivoire"),
 ('French Polynesia', 'France'),
 (nan, 'Mayotte'),
 ('Guangxi', 'China'),
 ('Grand Princess', 'Canada'),
 (nan, 'Mauritius'),
 (nan, 'Nigeria'),
 (nan, 'Puerto Rico'),
 ('Massachusetts', 'US'),
 ('Wyoming', 'US'),
 (nan, 'Oman'),
 ('Alberta', 'Canada'),
 (nan, 'Uzbekistan'),
 (nan, 'Bhutan'),
 (nan, 'Russia'),
 (nan, 'Rw

In [14]:
train['Date'].min(),train['Date'].max()

('2020-01-22', '2020-03-22')

In [16]:
test['Date'].max(),test['Date'].min()



('2020-04-23', '2020-03-12')

In [11]:
submission = test[["ForecastId"]]
submission.insert(1, "ConfirmedCases", 0)
submission.insert(2, "Fatalities", 0)

In [17]:
#final evaluation
train_start_date = "2020-01-12"
last_train_date = "2020-03-22"
test_end_date  = "2020-03-26"

for loc in locations:
    if type(loc[0]) is float and np.isnan(loc[0]):
        confirmed=train[((train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["ConfirmedCases"].values[0]
        deaths=train[((train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["Fatalities"].values[0]
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]<=test_end_date)), "ConfirmedCases"] = confirmed
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]<=test_end_date)), "Fatalities"] = deaths
    else:
        confirmed=train[((train["Province/State"]==loc[0]) & (train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["ConfirmedCases"].values[0]
        deaths=train[((train["Province/State"]==loc[0]) & (train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["Fatalities"].values[0]
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]<=test_end_date)), "ConfirmedCases"] = confirmed
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]<=test_end_date)), "Fatalities"] = deaths

submission

,ForecastId,ConfirmedCases,Fatalities
0,1,40.0,1.0
1,2,40.0,1.0
2,3,40.0,1.0
3,4,40.0,1.0
4,5,40.0,1.0
...,...,...,...
12207,12208,3.0,0.0
12208,12209,3.0,0.0
12209,12210,3.0,0.0
12210,12211,3.0,0.0


In [19]:
last_train_date = max(train["Date"])

for loc in locations:
    if type(loc[0]) is float and np.isnan(loc[0]):
        confirmed=train[((train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["ConfirmedCases"].values[0]
        deaths=train[((train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["Fatalities"].values[0]
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]>test_end_date)), "ConfirmedCases"] = confirmed
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]>test_end_date)), "Fatalities"] = deaths
    else:
        confirmed=train[((train["Province/State"]==loc[0]) & (train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["ConfirmedCases"].values[0]
        deaths=train[((train["Province/State"]==loc[0]) & (train["Country/Region"]==loc[1]) & (train["Date"]==last_train_date))]["Fatalities"].values[0]
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]>test_end_date)), "ConfirmedCases"] = confirmed
        submission.loc[((test["Country/Region"]==loc[1]) & (test["Date"]>test_end_date)), "Fatalities"] = deaths

submission

,ForecastId,ConfirmedCases,Fatalities
0,1,40.0,1.0
1,2,40.0,1.0
2,3,40.0,1.0
3,4,40.0,1.0
4,5,40.0,1.0
...,...,...,...
12207,12208,3.0,0.0
12208,12209,3.0,0.0
12209,12210,3.0,0.0
12210,12211,3.0,0.0


In [20]:
submission.to_csv("submission.csv", index=False)